In [1]:
import torch
import torch_directml
import glob
import numpy as np
from PIL import Image
import os
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import copy
from tqdm.auto import tqdm
import glob
import time

In [2]:
dml = torch_directml.device()
dml

device(type='privateuseone', index=0)

In [3]:
train_transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize(size = (224, 224)),
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees = 15),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


test_and_val_transform = transforms.Compose([
        # transforms.ToPILImage(),
        transforms.Grayscale(),
        transforms.Resize(size = (224, 224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [4]:
dataset_path = "../splitted"
train_dataset = dsets.ImageFolder(root=os.path.join(dataset_path, "train"), transform=train_transform)
test_dataset = dsets.ImageFolder(root=os.path.join(dataset_path, "test"), transform=test_and_val_transform)
val_dataset = dsets.ImageFolder(root=os.path.join(dataset_path, "val"), transform=test_and_val_transform)

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, drop_last=True, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, drop_last=True, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, drop_last=True, shuffle=False)

In [6]:
dataloaders = {"train" : train_loader,
               "val" : val_loader}

In [7]:
class MyCNN1(nn.Module):
    def __init__(self):
        super(MyCNN1, self).__init__()
        self.cnn_layer_1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), stride=1, padding=0, bias=False)
        self.batch_norm_1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.maxpool_1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.cnn_layer_2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=1, padding=0, bias=False)
        self.batch_norm_2 = nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.maxpool_2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.cnn_layer_3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=1, padding=0, bias=False)
        self.batch_norm_3 = nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.maxpool_3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)


        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.linear_layer_1 = nn.Linear(256*25*25, 2)
                                        
                        
    
    def forward(self, x):
        x = self.cnn_layer_1(x)
        x = self.batch_norm_1(x)
        x = self.relu(x)
        x = self.maxpool_1(x)


        x = self.cnn_layer_2(x)
        x = self.batch_norm_2(x)
        x = self.relu(x)
        x = self.maxpool_2(x)

        x = self.cnn_layer_3(x)
        x = self.batch_norm_3(x)
        x = self.relu(x)
        x = self.maxpool_3(x)
        
#         print(x.shape)

        x = self.flatten(x)

        x = self.linear_layer_1(x)
        
        return x

In [8]:
base_model = MyCNN1().to(dml)
base_model

MyCNN1(
  (cnn_layer_1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool_1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn_layer_2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (batch_norm_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool_2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn_layer_3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (batch_norm_3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool_3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU()
  (linear_layer_1): Linear(in_features=160000, out_features=2, bias=True)
)

In [9]:
num_classes = 2
num_epochs = 10
learning_rate = 1e-5
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(base_model.parameters(), lr=learning_rate)

In [10]:
validation_loss = []
training_loss = []

def train_model():
    since = time.time()
    val_acc_history = []
    
    best_model_wts = copy.deepcopy(base_model.state_dict())
    best_acc = 0.0
    
    progress_bar_train = tqdm(range(num_epochs * len(train_loader)))
    progress_bar_eval = tqdm(range(num_epochs * len(val_loader)))
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                base_model.train()  # Set model to training mode
            else:
                base_model.eval()   # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                # labels = torch.tensor([l.item() for l in label])
                inputs = inputs.to(dml)
                labels = labels.to(dml)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): 
                    outputs = base_model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        progress_bar_train.update(1)
                    elif phase == 'val':
                        progress_bar_eval.update(1)
                        
                running_loss += loss.item() * inputs.size(0)
                preds = preds.cpu()
                labels = labels.data.cpu()
                running_corrects += (preds == labels).sum()
            
            print("Lenght: ", len(dataloaders[phase].dataset))
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = float(running_corrects) / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'train':
              training_loss.append(epoch_loss)
            elif phase == 'val':
              validation_loss.append(epoch_loss)
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(base_model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    base_model.load_state_dict(best_model_wts)
    return base_model, val_acc_history

In [11]:
best_model, validation_acc_hist = train_model()

  0%|          | 0/43090 [00:00<?, ?it/s]

  0%|          | 0/6150 [00:00<?, ?it/s]

Epoch 0/9
----------
Lenght:  34478
train Loss: 0.6496 Acc: 0.6618
Lenght:  4926
val Loss: 0.5469 Acc: 0.7119

Epoch 1/9
----------
Lenght:  34478
train Loss: 0.5783 Acc: 0.7070
Lenght:  4926
val Loss: 0.4961 Acc: 0.7491

Epoch 2/9
----------
Lenght:  34478
train Loss: 0.5486 Acc: 0.7262
Lenght:  4926
val Loss: 0.4711 Acc: 0.7728

Epoch 3/9
----------
Lenght:  34478
train Loss: 0.5184 Acc: 0.7435
Lenght:  4926
val Loss: 0.5047 Acc: 0.7365

Epoch 4/9
----------
Lenght:  34478
train Loss: 0.4967 Acc: 0.7578
Lenght:  4926
val Loss: 0.4576 Acc: 0.7911

Epoch 5/9
----------
Lenght:  34478
train Loss: 0.4811 Acc: 0.7701
Lenght:  4926
val Loss: 0.5011 Acc: 0.7651

Epoch 6/9
----------
Lenght:  34478
train Loss: 0.4648 Acc: 0.7786
Lenght:  4926
val Loss: 0.4946 Acc: 0.7450

Epoch 7/9
----------
Lenght:  34478
train Loss: 0.4488 Acc: 0.7871
Lenght:  4926
val Loss: 0.4147 Acc: 0.8011

Epoch 8/9
----------
Lenght:  34478
train Loss: 0.4340 Acc: 0.7944
Lenght:  4926
val Loss: 0.4248 Acc: 0.7952

E

In [12]:
model_path = "../trained-models/mycnn.pt"
torch.save(best_model.state_dict(), model_path)